In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from prompts.classification_prompt import classification_prompt
from agents.classification_agent import classify_rfx

c:\Users\prerit.d.jain\OneDrive - Accenture\JTI\python_codes\rfx-chatbot-demo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import fitz
from llm_calling import llm_calling
from langchain_core.documents import Document
from data_preprocessing import data_preprocessing
import os

In [3]:
def file_names(directory):
    files=[]
    for file in os.listdir(directory):
        if file.endswith(".pdf"):
            files.append(os.path.join(directory, file))
    return files

In [4]:
def langchain_doc_creation(file_path):
    doc=[]
    for f in file_path:
        dp = data_preprocessing(file_path=f)
        doc += dp.load_data()
    return doc

In [5]:
directory = "./User_File"
file_path = file_names(directory)
doc_input = langchain_doc_creation(file_path)

In [6]:
doc_txt=''
for d in doc_input:
    doc_txt+= d.page_content
    doc_txt+= "\n\n"

In [7]:
system_prompt = classification_prompt

In [8]:
rfx_type = classify_rfx(text=doc_txt,model_name="qwen2.5:7b").classify_rfx_solve()

[TIMING] RAG classification took 452.55s


In [9]:
rfx_type

{'rfx_type': 'RFP'}

In [12]:
from creating_retriever import universal_retrieval,user_retriever
from langchain_qdrant import QdrantVectorStore
from llm_calling import llm_calling
from pathlib import Path

In [ ]:
embeddings = llm_calling(embedding_model="llama3.2:latest").call_embed_model()

## User Retrieval
type_of_retrieval = "dense" #@param ["dense", "sparse", "hybrid"]
collection_name = f"""jti_rfp_{type_of_retrieval}_user"""
path = f"""./tmp/langchain_qdrant_user_{type_of_retrieval}"""
my_file = Path(path+f"""/collection/{collection_name}/storage.sqlite""")
directory = "./User_File"
#directory = "./Input_Files"

if my_file.is_file():
    print("DB Exists")
    retriever_input = universal_retrieval(collection_name=collection_name,embeddings=embeddings,path=path).load_existing_vdb_collection()
else:
    print("DB does not exist")